## Set the path of where the csv files are located, as well as initial import statements

In [1]:
csv_data_path = "/Users/quinton/Documents/Projects/COMET_Prestudy/scripts/collection/formal_scripts/output_data_RIPE_01_2017.csv" # **PLACE CSV FILEPATH HERE**
sep = "\t" # csv delimiter used
small_model = False # use smaller ML model for better performance, less accuracy

In [2]:
from pathlib import Path 
import pandas as pd
import numpy as np

# Gensim imports
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy
import spacy
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# visualization
import pyLDAvis
import pyLDAvis.gensim_models

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/quinton/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/Users/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


## Functions to handle data loading and data writing

In [3]:
def load_data(filename) -> pd.DataFrame:
    filename = Path(filename)
    if not filename.is_file():
        raise FileNotFoundError

    with open(filename.resolve(), 'r', encoding='utf-8') as file:
        data = pd.read_csv(file, sep=sep)
    return data

def load_data_by_year(directory: Path) -> pd.DataFrame:
    for data_file in Path.glob(Path.joinpath(directory, '*.csv')):
            yield load_data(data_file.resolve())

def write_data(filename: Path, data: pd.DataFrame):
    with open(filename.as_uri(), 'w', encoding='utf-8') as f:
        data.to_csv(f, sep=sep)


In [4]:
# obtain the stop words
stopwords = stopwords.words("english")
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [5]:
data = load_data(csv_data_path)['content']
data.head()

0    Hi,just a question:any interest to have a prob...
1    Hi Meikel,Sorry for the silence. I had noticed...
2    Hi everyone,accoring to this site, a CentOS 7 ...
3    Hi,I may be wrong but it looks like there are ...
4    Hello everyone, I would like to host a new Atl...
Name: content, dtype: object

# Content Preprocessing

In [6]:
def lemmatize(posts: pd.DataFrame, allowed_post_tags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm" if small_model else "en_core_web_trf", disable=["parser", 'ner'])
    texts_out = []

    for post in posts:
        doc = nlp(post)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_post_tags:
                new_text.append(token.lemma_)
        final = ' '.join(new_text)
        texts_out.append(final)
    return texts_out

lemmatized_data = lemmatize(data.astype(str))
lemmatized_data[0][:10]

/Users/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
Token indices sequence length is longer than the specified maximum sequence length for this model (733 > 512). Running this sequence through the model will result in indexing errors


'just quest'

In [7]:
def tokenize_lemma(posts: list):
    final = []
    for post in posts:
        new = gensim.utils.simple_preprocess(post, deacc=True)
        final.append(new)
    return final

tokenized_data = tokenize_lemma(lemmatized_data)
data = [[word for word in phrase if word not in stopwords] for phrase in tokenized_data]
data

[['question',
  'interest',
  'probe',
  'sailboat',
  'officially',
  'supported',
  'get',
  'probe',
  'fix',
  'home',
  'usb',
  'stick',
  'ro',
  'dead',
  'wonder',
  'leave',
  'home',
  'take',
  'yes',
  'api',
  'way',
  'update',
  'location',
  'nice',
  'moving',
  'best',
  'mobile',
  'id'],
 ['sorry',
  'silence',
  'notice',
  'issue',
  'apparently',
  'forget',
  'reply',
  'look',
  'bear',
  'respond',
  'soon',
  'update',
  'regard',
  'dot',
  'write',
  'really',
  'one',
  'idea',
  'thought',
  'concern',
  'problem',
  'appreciate',
  'stud',
  'uni',
  'dot',
  'send',
  'ripe',
  'dot',
  'net',
  'ripe',
  'ripe',
  'dot',
  'net',
  'subject',
  'ongoing',
  'problem',
  'measurement',
  'dear',
  'mailing',
  'context',
  'final',
  'thesis',
  'conduct',
  'research',
  'relate',
  'use',
  'probe',
  'preparation',
  'main',
  'measurement',
  'issue',
  'around',
  'probe',
  'dns',
  'query',
  'measurement',
  'id',
  'standard',
  'measurement',

# Generate bigrams/trigrams

In [8]:
bigrams_phrases = gensim.models.Phrases(data, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigrams_phrases[data], threshold=100)

bigram = gensim.models.phrases.Phraser(bigrams_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return (bigram[doc] for doc in texts)

def make_trigrams(texts):
    return (trigram[bigram[doc]] for doc in texts)

data_bigrams = make_bigrams(data)
data_bigrams_trigrams = make_trigrams(data_bigrams)
bgtg = list(data_bigrams_trigrams)
bgtg

[['question',
  'interest',
  'probe',
  'sailboat',
  'officially',
  'supported',
  'get',
  'probe',
  'fix',
  'home',
  'usb_stick',
  'ro',
  'dead',
  'wonder',
  'leave',
  'home',
  'take',
  'yes',
  'api',
  'way',
  'update',
  'location',
  'nice',
  'moving',
  'best',
  'mobile',
  'id'],
 ['sorry',
  'silence',
  'notice',
  'issue',
  'apparently',
  'forget',
  'reply',
  'look',
  'bear',
  'respond',
  'soon',
  'update',
  'regard',
  'dot',
  'write',
  'really',
  'one',
  'idea',
  'thought',
  'concern',
  'problem',
  'appreciate',
  'stud',
  'uni',
  'dot',
  'send',
  'ripe',
  'dot',
  'net',
  'ripe',
  'ripe',
  'dot',
  'net',
  'subject',
  'ongoing',
  'problem',
  'measurement',
  'dear',
  'mailing',
  'context',
  'final',
  'thesis',
  'conduct',
  'research',
  'relate',
  'use',
  'probe',
  'preparation',
  'main',
  'measurement',
  'issue',
  'around',
  'probe',
  'dns',
  'query',
  'measurement',
  'id',
  'standard',
  'measurement',
  'i

# TF-IDF non-useful common word removal

In [9]:
from gensim.models import TfidfModel

id2word = corpora.Dictionary(bgtg)
corpus = [id2word.doc2bow(text) for text in bgtg]

corpus

tfidf = TfidfModel(corpus=corpus, id2word=id2word, )

low_value = 0.03
words = []
words_missing_tfidf = []
for iter in range(0,len(corpus)):
    bow = corpus[iter]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_tfidf = [id for id in bow_ids if id not in tfidf_ids]

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_tfidf]
    corpus[iter] = new_bow

# Generate id2word dictionary

In [10]:
# deprecated since tf-idf implementation
# id2word = corpora.Dictionary(tokenized_data)

# corpus = []

# for text in tokenized_data:
#     new = id2word.doc2bow(text)
#     corpus.append(new)

# corpus[0][:10]

# LDA Visualization

In [13]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=7,
random_state=100, update_every=1, chunksize=100, passes=10, alpha="auto")

In [14]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds='mmds', R=30)
vis

/Users/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/Users/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecat

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.276243  0.135882       1        1  26.837268
5     -0.023563 -0.206919       2        1  21.855046
2     -0.235465 -0.178271       3        1  17.830412
1     -0.048156  0.184385       4        1  16.009298
4      0.212852 -0.173532       5        1   8.515426
0      0.217510  0.046131       6        1   4.643437
6      0.153064  0.192325       7        1   4.309113, topic_info=            Term        Freq       Total Category  logprob  loglift
52   measurement  386.000000  386.000000  Default  30.0000  30.0000
74          send  110.000000  110.000000  Default  29.0000  29.0000
404       credit  126.000000  126.000000  Default  28.0000  28.0000
40           dot  101.000000  101.000000  Default  27.0000  27.0000
94        anchor  112.000000  112.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
329      session    3.370491    6.445232   Topic7  -5.5623   2.4962
131         host    4.027744   72.315205   Topic7  -5.3842   0.2566
103         plan    3.535561   15.866048   Topic7  -5.5145   1.6431
48          look    3.738479   66.199923   Topic7  -5.4587   0.2704
180         help    3.511371   35.732847   Topic7  -5.5214   0.8244

[375 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
215       1  0.029948       able
215       2  0.119793       able
215       3  0.299483       able
215       4  0.539069       able
434       2  0.164160    account
...     ...       ...        ...
120       3  0.982370      wrong
113       4  0.732082       year
113       7  0.209166       year
154       5  0.983302  yesterday
1388      5  0.913818        yet

[526 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 6, 3, 2, 5, 1, 7])